# Handling raw data from Illumina

For the first part we will work with the [10X Genomics 5K Neuron data set](https://support.10xgenomics.com/single-cell-gene-expression/datasets/6.0.0/SC3_v3_NextGem_DI_Neurons_5K_Multiplex?). This dataset includes ~5,000 cells collected from E18 mouse  combined cortex, hippocampus, and subventricular zone (C57BL/6 strain). This was part of the demostration of CellPlex (v3.1 Chemistry), in which data was analyzed using CellRanger 6.0.0. The summary results can be found [here](https://cf.10xgenomics.com/samples/cell-exp/6.0.0/SC3_v3_NextGem_DI_Neurons_5K_SC3_v3_NextGem_DI_Neurons_5K/SC3_v3_NextGem_DI_Neurons_5K_SC3_v3_NextGem_DI_Neurons_5K_web_summary.html).

## Download

*This part of the code is very time consuming, so we will not run it during the workshop.*

The most straigth forward way to analyze data generated with Chromium Single Cell Gene Expression technologies is using the [CellRanger software](https://support.10xgenomics.com/single-cell-gene-expression/software/overview/welcome). A very similar pipeline to do this is [STARsolo](https://github.com/alexdobin/STAR/blob/master/docs/STARsolo.md), which is the single-cell extension of the STAR aligner. STARsolo also does the filtering, mapping, demultiplexing and quantification steps internally and outputs a feature counts matrix. Since we're using a reference data set from 10X genomics, we will use CellRanger.

To begin the analysis we need two things:

1. Reference genome annotation 
2. *fastq* files from the RNAseq experiment


**Reference for mm10**

>```bash
REF=../references
cd ${REF}
>wget https://cf.10xgenomics.com/supp/cell-exp/refdata-gex-mm10-2020-A.tar.gz
tar -xvzf refdata-gex-mm10-2020-A.tar.gz
rm refdata-gex-mm10-2020-A.tar.gz
>```


**Fastq files**

>```bash
FASTQ=../fastq/neurons5k
cd ${FASTQ}
>wget https://cf.10xgenomics.com/samples/cell-exp/6.0.0/SC3_v3_NextGem_DI_Neurons_5K_Multiplex/SC3_v3_NextGem_DI_Neurons_5K_Multiplex_fastqs.tar 
tar -xvf SC3_v3_NextGem_DI_Neurons_5K_Multiplex_fastqs.tar
rm SC3_v3_NextGem_DI_Neurons_5K_Multiplex_fastqs.tar
>```

# Counting features

We downloaded the *fastq* files directly from the 10X website, so we do not need to run the first step of the pipeline `cellranger mkfastq`. If you are downloading data from [NCBI's SRA](https://www.ncbi.nlm.nih.gov/sra) you can check the [documentation](https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/using/count) for this step.

## Run CellRanger count

All we need to do is run `cellranger count` to get our feature counts. If you are runnninng this in a cluster that uses the workload manager Slurm, you can use the following job script changing `REF` to the path containing the reference genome and `FASTQ` to the directory containing your *fastq* files.

>```bash
#!/usr/bin/bash
#SBATCH --cpus-per-task=16
#SBATCH --mem=60gb
#SBATCH --time=02:00:00
#SBATCH --job-name=cellranger
#SBATCH --err=logs/err.cranger.%j.log
#SBATCH --output=logs/out.cranger.%j.log
>
>FASTQ=../fastq/neurons5k/SC3_v3_NextGem_DI_Neurons_5K/SC3_v3_NextGem_DI_Neurons_5K_gex/
>REF=../references/refdata-gex-mm10-2020-A
>OUT=../cellranger/neurons5k
>
>cellranger count --id=${OUT} \
                 --transcriptome=${REF} \
                 --fastqs=${FASTQ} \
                 --sample=SC3_v3_NextGem_DI_Neurons_5K_gex \
                 --expect-cells=5000 \
                 --localcores=16 \
                 --localmem=60              
>```

## Exploring the output

We will find the outputs of this step inside `${OUT}/outs` and it will look like this:

>```bash
analysis
filtered_feature_bc_matrix
metrics_summary.csv
possorted_genome_bam.bam
raw_feature_bc_matrix 
web_summary.html
cloupe.cloupe
filtered_feature_bc_matrix.h5
molecule_info.h5
possorted_genome_bam.bam.bai  
raw_feature_bc_matrix.h5
>```




### Report

A detailed description of the output files can be found in the [documentation](https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/output/overview#count).   

For now, we can look at the [web summary](../output/cellranger/web_summary.html) to see the statistics of the different pre-processing steps as well as some preeliminary analysis.

### Counts 

For the next activity, we will use the files in `filtered_feature_bc_matrix` as input for our analysis. This folder will have the following files:

>```
barcodes.tsv.gz
features.tsv.gz
matrix.mtx.gz
>```

They contain the cell barcodes (row names), the feature names (column names) and the cell by feature count matrix.

## Additional QCs

The CellRanger pipeline already calculates several quality control metrics for the reads and the alignments, but we will look at a few more. Some of the tools developed for bulk RNA seq are also useful for single-cell RNA seq such as [REseQC](http://rseqc.sourceforge.net).

This data was generated with a protocol that uses poly-A oligos to enrich for messenger RNAs, which causes a coverage bias towards the 3' end of genes.

The following code calculates the read coverage along different percentiles of gene bodies:

>```bash
#!/usr/bin/bash
># Activate RSeQC environment
RSeQCpath=/home/lmoral7/env/RSeQC/bin
source ${RSeQCpath}/activate
BED=/project/6007998/lmoral7/references/hg38_RefSeq.bed
BAM=../cellranger/neurons5k/outs/possorted_genome_bam.bam
OUT=../output/rseqc/neurons5K.txt
geneBody_coverage.py -i ${BAM} -r ${BED} -o ${OUT}
>```
